In [ ]:
!pip install datasets transformers
!apt install git-lfs


In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset, load_metric
import datasets as ds
from pynvml import *
from transformers import default_data_collator,AutoTokenizer
from tqdm.auto import tqdm
import collections
import numpy as np 

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # We need to take care of contexts that are too long and were thus split into several entries
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

# NOTE: This function is part of the official evaluating script of SQuAD v1.1
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
    # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]
        
    return predictions

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Parameters for SQuAD training**

In [ ]:
model_checkpoint = "/content/gdrive/MyDrive/thesis/baseline_intermediate/train_squad_en/"
batch_size = 32
categories = ['xquad.ar', 'xquad.de', 'xquad.zh', 'xquad.vi', 'xquad.en', 'xquad.es', 'xquad.hi', 'xquad.el', 'xquad.th', 'xquad.tr', 'xquad.ru', 'xquad.ro']
datasets = load_dataset("squad")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_length = 384 
doc_stride = 128 
max_answer_length = 30
pad_on_right = tokenizer.padding_side == "right"
data_collator = default_data_collator
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# **Training Configuration**

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoModelForSequenceClassification

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to("cuda")

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    push_to_hub=False,
    fp16 = True
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation.en"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train()
trainer.save_model("test-squad-trained")

In [ ]:
!cp -r '/content/test-squad-trained' '/content/gdrive/MyDrive/thesis/trained_intermediate/nerml_mlqa'

In [ ]:
max_answer_length = 30

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoModelForSequenceClassification
trainer = Trainer(
    model,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
languages = ['ar','de','es','vi','zh','hi']
datasets = {}
for language in languages:
  datasets[language] = load_dataset("squad","mlqa-translate-test."+language)


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoModelForSequenceClassification,AutoTokenizer

model_checkpoints = ["/content/gdrive/MyDrive/thesis/baseline_downstream/mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/xquad_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/nerml_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/mnli_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa",
                    ]
languages = ['en']

results = {}
for model_checkpoint in model_checkpoints:
  results[model_checkpoint] = {}
  model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to("cuda")
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  trainer = Trainer(
      model,
      tokenizer=tokenizer,
  )
  for language in languages:
    validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
    ) 
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
    metric = load_metric("squad")
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
    result = metric.compute(predictions=formatted_predictions, references=references)  
    results[model_checkpoint][language] = result


loading configuration file /content/gdrive/MyDrive/thesis/baseline_downstream/mlqa/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/thesis/baseline_downstream/mlqa",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 119547
}

loading weights file /content/gdrive/MyDrive/thesis/baseline_downstream/mlqa/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from 

  0%|          | 0/11 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10851
  Batch size = 8


Post-processing 10570 example predictions split into 10851 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

...

loading configuration file /content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

loading weights file /content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBert

  0%|          | 0/11 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10784
  Batch size = 8


Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoModelForSequenceClassification,AutoTokenizer

model_checkpoints = ["/content/gdrive/MyDrive/thesis/baseline_downstream/mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/xquad_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/nerml_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/mnli_mlqa",
                     "/content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa",
                    ]
languages = ['ar','de','es','vi','zh','hi']

results = {}
# For every language of the dataset, load the model and execute inference on the validation set
for model_checkpoint in model_checkpoints:
  results[model_checkpoint] = {}
  model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to("cuda")
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  trainer = Trainer(
      model,
      tokenizer=tokenizer,
  )
  for language in languages:
    validation_features = datasets[language]["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets[language]["test"].column_names
    ) 
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    # Apply post-processing to infered values
    final_predictions = postprocess_qa_predictions(datasets[language]["test"], validation_features, raw_predictions.predictions)
    metric = load_metric("squad")
    # Apply IDs to predictions, and format the output in the expected way
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets[language]["test"]]
    # Compute the F1 score with the formatted predictions
    result = metric.compute(predictions=formatted_predictions, references=references)  
    results[model_checkpoint][language] = result


loading configuration file /content/gdrive/MyDrive/thesis/baseline_downstream/mlqa/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/thesis/baseline_downstream/mlqa",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 119547
}

loading weights file /content/gdrive/MyDrive/thesis/baseline_downstream/mlqa/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from 

  0%|          | 0/6 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 6239
  Batch size = 8


Post-processing 5335 example predictions split into 6239 features.


  0%|          | 0/5335 [00:00<?, ?it/s]

...

  0%|          | 0/5137 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5491
  Batch size = 8


Post-processing 4918 example predictions split into 5491 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

In [ ]:
results

{'/content/gdrive/MyDrive/thesis/baseline_downstream/mlqa': {'en': {'exact_match': 68.07000946073794,
   'f1': 77.97300960557064}},
 '/content/gdrive/MyDrive/thesis/trained_intermediate/mnli_mlqa': {'en': {'exact_match': 67.97540208136235,
   'f1': 77.6118748170819}},
 '/content/gdrive/MyDrive/thesis/trained_intermediate/nerml_mlqa': {'en': {'exact_match': 67.92809839167455,
   'f1': 77.45650234421817}},
 '/content/gdrive/MyDrive/thesis/trained_intermediate/xnli_mlqa': {'en': {'exact_match': 71.5515610217597,
   'f1': 81.27773454146448}},
 '/content/gdrive/MyDrive/thesis/trained_intermediate/xquad_mlqa': {'en': {'exact_match': 67.7578051087985,
   'f1': 77.52851613790993}}}

In [ ]:
for result in results.keys():
  print(np.mean([results[result][lan]["exact_match"] for lan in languages]))

26.40507528873003
26.1663193818179
26.31335636882301
26.575787349819745
29.09649565358851
